# 프로젝트 : 멋진 작사가 만들기

### 라이브러리 버전 확인

In [1]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow as tf

print(tf.__version__)

2.6.0


### Step 1. 데이터 다운로드


### Step 2. 데이터 읽어오기

In [2]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장

txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof']


### Step 3. 데이터 정제

In [3]:
import re
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


# 이 문장이 어떻게 필터링되는지 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛰기
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split(' ')) <=15 :
        corpus.append(preprocessed_sentence)
    
# 정제된 결과를 30개 확인
corpus[:30]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>',
 '<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>',
 '<start> she tied you <end>',
 '<start> to a kitchen chair <end>',
 '<start> she broke your throne , and she cut your hair <end>',
 '<start> and from your lips she drew the hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah you say i took the name in vain <end>',
 '<start> i don t even know the name <end>',
 '<start> th

In [5]:
len(corpus)

156013

In [6]:
# print(len(corpus[0].split(' ')))

In [7]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer
    # 우리는 이미 문장을 정제했으니 filters가 필요없다
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꾼다
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성한다
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환한다
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰주기
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰주기
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f2e3811bee0>


In [8]:
print(tensor[:3, :10]) # 행 , 렬

[[   2   50    4   95  303   62   53    9  946 6263]
 [   2   15 2967  871    5    8   11 5739    6  374]
 [   2   33    7   40   16  164  288   28  333    5]]


In [9]:
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [10]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

# 행 뒤쪽에 0이 많은 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 
# 0으로 패딩(padding)을 채워 넣은 것
# 0은 바로 패딩 문자 <pad>

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


In [11]:
print(len(src_input))
print(len(tgt_input))

156013
156013


### Step 4. 평가 데이터셋 분리

In [12]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input, 
                                                          test_size = 0.2, 
                                                          shuffle=True, 
                                                          random_state=10)

In [13]:
# print(enc_train)
# print(enc_val)
# print(dec_train)
# print(dec_val)

print(len(enc_train))
print(len(enc_val))
print(len(dec_train))
print(len(dec_val))

124810
31203
124810
31203


### Step 5. 인공지능 만들기

In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 7000개로 선언했기 때문에, tokenizer.num_words의 값은 7000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만든다
# 데이터셋에 대해서는 아래 문서를 참고
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [15]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져온다   
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문

In [16]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어보기
model(src_sample)


<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-8.93012184e-05,  7.20549506e-05,  9.29457528e-05, ...,
          2.51660647e-04, -7.69653052e-05, -5.14737367e-05],
        [-7.50735635e-05, -2.91304255e-04,  1.68191458e-04, ...,
          2.52558821e-04,  1.92126827e-04, -4.31698863e-04],
        [-1.21092344e-04, -3.92713468e-04,  2.71301251e-04, ...,
          8.04730880e-05,  4.00504650e-04, -4.79042967e-04],
        ...,
        [ 1.27847143e-03, -2.89400370e-04, -1.62393306e-04, ...,
          6.40396902e-04,  1.37764553e-03,  7.25694757e-04],
        [ 1.33159105e-03,  1.01393474e-04, -3.08291928e-04, ...,
          7.24932994e-04,  1.71499804e-03,  9.37055796e-04],
        [ 1.31831015e-03,  4.65723162e-04, -5.00603230e-04, ...,
          1.00981188e-03,  1.51096901e-03,  1.24855794e-03]],

       [[-8.93012184e-05,  7.20549506e-05,  9.29457528e-05, ...,
          2.51660647e-04, -7.69653052e-05, -5.14737367e-05],
        [-5.57078929e-05,  1.41050434e-04,  3

In [17]:
# 모델의 구조를 확인
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [18]:
optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수
    from_logits=True,
    reduction='none' # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정

In [19]:

model.fit(dataset, epochs=30)
#Loss
# tf.keras.losses.SparseCategoricalCrossentropy : https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none') # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.

Epoch 1/30
609/609 [==============================] - 96s 151ms/step - loss: 3.4471
Epoch 2/30
609/609 [==============================] - 99s 163ms/step - loss: 2.9594
Epoch 3/30
609/609 [==============================] - 99s 162ms/step - loss: 2.7828
Epoch 4/30
609/609 [==============================] - 99s 163ms/step - loss: 2.6493
Epoch 5/30
609/609 [==============================] - 99s 162ms/step - loss: 2.5354
Epoch 6/30
609/609 [==============================] - 99s 163ms/step - loss: 2.4321
Epoch 7/30
609/609 [==============================] - 99s 163ms/step - loss: 2.3372
Epoch 8/30
609/609 [==============================] - 99s 163ms/step - loss: 2.2495
Epoch 9/30
609/609 [==============================] - 99s 163ms/step - loss: 2.1668
Epoch 10/30
609/609 [==============================] - 99s 163ms/step - loss: 2.0888
Epoch 11/30
609/609 [==============================] - 99s 163ms/step - loss: 2.0147
Epoch 12/30
609/609 [==============================] - 99s 163ms/step - lo

In [20]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [21]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20) # 시작문장으로 he를 넣어 문장생성 함수 실행

'<start> i love you so much <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> i am", max_len=20) 

'<start> i am a god <end> '

In [23]:
generate_text(model, tokenizer, init_sentence="<start> i like", max_len=20) 

'<start> i like the way how you re touchin me <end> '

In [24]:
generate_text(model, tokenizer, init_sentence="<start> my", max_len=20) 

'<start> my girl went through my cell phone <end> '

In [25]:
generate_text(model, tokenizer, init_sentence="<start> you", max_len=20) 

'<start> you re the only power <end> '

# 회고

- 이번 프로젝트에서 **어려웠던 점,**

이전 fund에서 NLP에대해 처음접해보았는데, 개념부터 너무 어려웠었다.

model fit을 하는데 시간이 너무 오래걸려서 힘들었다.




- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.

함수와 코드를 하나하나보면서 이해했어야했는데, 그러지 못한 상태로 마무리를 하게되어 아쉽다.

자연어 처리를 위해서 정규표현식을 했는데, 정규표현식을 좀 더 공부해야겠다.


- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.

특수문자 제거, 토크나이저 생성, 토큰화 했을때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외시키기, 단어장크기 12,000장 이상, 평가데이터셋 총 데이터의 20%, 

10 Epoch 안에 val_loss값 2.2 줄여보기

Epoch 10/30
609/609 [==============================] - 99s 163ms/step - loss: 2.0888

텍스트 제너레이션 결과로 생성된 문장이 해석가능한 문장인지 살펴보기



- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정.느낌

이전 노드의 내용을 그대로 참고해 학습부분에서 스스로 고민해본것이 부족한점

- **자기 다짐**

관심있는 분야와 그렇지 않은 분야를 정해놓고 어느 한쪽에 집중하는건 맞지만 

학습에서도 소홀히 하면 결국 나에게 좋을건 없다는걸 다시한번 더 깨닳았다.

Embedding 개념을 다시 한번 살펴보고 이번 프로젝트와 19번 fund의 내용을 익힐 수 있게 노력해야겠다.